In [26]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Layer,Input,Concatenate, Conv2D, Embedding, MaxPool2D, Flatten, Dropout, Dense, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt

from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint


In [42]:
def build_discriminator(in_shape=(32,32,3), n_labels= 10):      # cifair image shape is 32x32x3
    
    
    # initialize the input layer
    input_layer= Input(shape= (1,))   
    
    # embedding for categorical input each label (total 10 classes for cifar), will be represented by a vector 
    # of size 50. This vector of size 50 will be learnt by the discriminator
    in_labels= Embedding(n_labels, 50)(input_layer) #Shape 1,50 scale up to image dimensions with linear activatio
    
    n_nodes= in_shape[0] * in_shape[1]    #32x32 = 1024.
    
    layer_2= Dense(n_nodes)(in_labels)
    layer_3= Reshape((in_shape[0],in_shape[1],1))(layer_2)
    
    in_image= Input(shape= in_shape)
    merge= Concatenate()([in_image, layer_3])  # n, 32, 32, 4 
    
      
    layer_4= Conv2D(128, (4,4), (2,2), padding= 'same')(merge)    # 16 X 16 x128
    layer_4= LeakyReLU(alpha= 0.2)(layer_4)
    
    layer_5= Conv2D(128, (4,4), (2,2), padding= 'same')(layer_4)  # 8x8x128
    layer_5= LeakyReLU(alpha= 0.2)(layer_5)

    layer_6= Flatten()(layer_5)              #8192  (8*8*128=8192)
    out_layer= Dense(1, activation= 'sigmoid')(layer_6)    # shape= 1
    
    
    model= Model([in_image, in_labels], out_layer)
    
    model.compile(optimizer='Adam',
                 loss= 'binary_crossentropy',
                 metrics= ['accuracy'])
    
    return model

In [43]:
test_discr = build_discriminator()
print(test_discr.summary())

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_32 (InputLayer)       [(None, 1, 50)]              0         []                            
                                                                                                  
 dense_19 (Dense)            (None, 1, 1024)              52224     ['input_32[0][0]']            
                                                                                                  
 input_31 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 reshape_13 (Reshape)        (None, 32, 32, 1)            0         ['dense_19[1][0]']            
                                                                                            

In [48]:
def build_generator(latent_dim, n_labels= 10):
    
        inp_labels= Input((1,))
        layer_1= Embedding(n_labels, 50)(inp_labels)
        
        n_nodes= 8*8                                 # To match the dimensions for concatenation later in this step. 
        
        layer_2= Dense(n_nodes)(layer_1)
        layer_3= Reshape((8,8,1))(layer_2)           # reshape so that we can add with the dense layer of size 8 x 8x 128
    
        inp_layer= Input((latent_dim,))              #Input of dimension 100
        
        n_nodes= 8*8*128                             #shape=8192
        
        li= Dense(n_nodes)(inp_layer)
        li = LeakyReLU(alpha=0.2)(li)
        li= Reshape((8, 8, 128))(li)                   # 8 x 8 x 128
        
        merge= Concatenate()([li, layer_3])         #Shape=8x8x129 (Extra channel corresponds to the label)
        
        layer_5= Conv2DTranspose(128, (4,4), (2,2), padding= 'same')(merge)   # 16 x 16 x128
        layer_5= LeakyReLU(alpha=(0.2))(layer_5)
        
        layer_6= Conv2DTranspose(128, (4,4), (2,2), padding= 'same')(layer_5)   # 32 X 32 X 128
        layer_6= LeakyReLU(alpha= 0.2)(layer_6)
        
        out_layer= Conv2D(3, (8, 8), activation= 'tanh', padding= 'same')(layer_6)  # 32 x 32 x 3
        
        model= Model([inp_layer, inp_labels], out_layer)
        
        return model
        

In [49]:
test_gen = build_generator(100, n_labels=10)
print(test_gen.summary())

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_36 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 input_35 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 dense_24 (Dense)            (None, 8192)                 827392    ['input_36[0][0]']            
                                                                                                  
 embedding_16 (Embedding)    (None, 1, 50)                500       ['input_35[0][0]']            
                                                                                            

In [66]:
def generate_gan(g_model, d_model, latent_dim):
    
    # do not train discriminator when traininf generator
    d_model.trainiable= False
    
    ## connect generator and discriminator...
    # first, get noise and label inputs from generator model
    gen_noise, gen_label= g_model.input()
    
    # get image output from the generator model
    gen_output= g_model.output()              # 32x32x3
    
    # generator image output and corresponding input label are inputs to discriminator
    dec_output= d_model(gen_output, gen_label)
    
    # define gan model as taking noise and label and outputting a classification
    model= Model([gen_noise, gen_label], dec_output)
    
    # compile the model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(optimizer= opt, 
                 loss= 'binary_crossentropy')
    
    return model

In [65]:
# load real images 

def load_real_samples():
    
    (X_train, y_train),(_, _)= keras.datasets.cifar10.load_data()
    
    # convert the images into float
    X_train = X_train.astype('float32')
    
    # Normalize the images
    X_train= (X_train-127.5)/127.5
    
    
    return [X_train, y_train]



In [63]:
def generate_real_samples(dataset, n_samples):
    
    # split into images and labels
    
    images, labels= dataset
    
    
    # choose random instances
    idx= np.random.randint(0, images.shape[0], n_samples)
    
    # select images and labels
    real_img, labels= images[idx], labels[idx]
    
    # generate class labels and assign to y (don't confuse this with the above labels that correspond to cifar labels)
    y_ones= np.ones((n_samples, 1))
    
    return [real_img, labels], y_ones
    

In [60]:
# generate  latent point

def generate_latent_point(latent_point, n_samples, n_classes= 10):
    
    # generate points in the latent space
    noise= np.random.normal(0, 1, (n_samples, latent_point))
    
    # generate the labels
    labels= randint(0, n_classes, n_samples)
    
    return [noise, labels]
    

In [61]:
# generate fake images

def generate_fake_images(latent_point, n_sample, generator):
    
    # generate latent points and labels
    latent_point, labels= generate_latent_point(latent_point, n_sample)
    
    # generate fake images
    fake_imgs= generator.predict([latent_point, labels])
    
    # create class labels
    y= np.zeros((n_sample, 1))
    
    return [fake_imgs, labels], y

In [62]:
# train the generator and discriminator
#We loop through a number of epochs to train our Discriminator by first selecting
#a random batch of images from our true/real dataset.
#Then, generating a set of images using the generator. 
#Feed both set of images into the Discriminator. 
#Finally, set the loss parameters for both the real and fake images, as well as the combined loss. 

def train(epochs, batch_size, g_model, d_model, gan_model, dataset, latent_point):
    
    batch_per_epoch= dataset.shape[0]/batch_size
    half_batch= int(batch_size/2)
    
    for epoch in range(epochs):
        for batch in range(batch_per_epoch):
            
            
            # generate real images
            [X_real, labels_real], y_real= generate_real_samples(dataset, half_batch)
            
            # train discriminator on the real images
            d_loss_real= d_model.train_on_batch([X_real, labels_real], y_real)
            
            # generate fake images
            [X_fake, labels_fake], y_fake= generate_fake_images(latent_point, half_batch, g_model)
            
            d_loss_fake= d_model.train_on_batch([X_fake, labels_fake], y_fake)
            
            # prepare points in latent space as input for the generator
            [z_input, label_input]= generate_latent_point(latent_point, half_batch)
            
            #The generator wants the discriminator to label the generated samples as valid (ones)
            #This is where the generator is trying to trick discriminator into believing
            #the generated image is true (hence value of 1 for y) create inverted labels for the fake samples
            
            y_fake= np.ones((batch_size,1))
            
            
            # Generator is part of combined model where it got directly linked with the discriminator
            # Train the generator with latent_dim as x and 1 as y. 
            # Again, 1 as the output as it is adversarial and if generator did a great
            #job of folling the discriminator then the output would be 1 (true)
            
            # update the generator via the discriminator's error
            
            valid= gan_model.train_on_batch([z_input, label_input], y_fake)
            
            
            # Print losses on this batch
            print('Epoch>%d, Batch%d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, batch_per_epoch, d_loss_real, d_loss_fake, g_loss))
           
            # save the generator model
            g_model.save('cifar_conditional_generator_25epochs.h5')
            

In [ ]:
# train the gan

# size of latent
latent_dim= 100

# create the driscriminator
d_model= build_discriminator()

# create generator
g_model= build_generator()

# create gan
gan_model = generate_gan(g_model, d_model, latent_dim)


# load images
dataset= load_real_samples()

# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=2)



In [ ]:
##########################################################
# Now, let us load the generator model and generate images
# Lod the trained model and generate a few images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
import numpy as np
# 

#Note: CIFAR10 classes are: airplane, automobile, bird, cat, deer, dog, frog, horse,
# ship, truck

# load model
model = load_model('cifar_conditional_generator_250epochs.h5')

# generate multiple images

latent_points, labels = generate_latent_points(100, 100)
# specify labels - generate 10 sets of labels each gping from 0 to 9
labels = asarray([x for _ in range(10) for x in range(10)])
# generate images
X  = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
X = (X*255).astype(np.uint8)
# plot the result (10 sets of images, all images in a column should be of same class in the plot)
# Plot generated images 
def show_plot(examples, n):
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :, :])
	plt.show()
    
show_plot(X, 10)